Imports

In [ ]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

import time

Etape 5: Entraînez un modèle word2vec (word embeddings) + adapter (éventuellement) les paramètres window (taille de la fenêtre) et min_count (nombre minimum d’occurrences d’un mot)


In [11]:
# Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]
            
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

In [12]:
# Création d'un corpus d'unigrams, bigrams, trigrams
bigram_phrases = Phrases(sentences)
bigram_phraser = Phraser(phrases_model=bigram_phrases)
trigram_phrases = Phrases(bigram_phraser[sentences])
trigram_phraser = Phraser(phrases_model=trigram_phrases)
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [23]:
# Entrainement du modèle word2vec
# %%time #==> ne marche pas

start1 = time.perf_counter()
start2 = time.process_time()
model1 = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé (5 mots avant + 5 mots après)
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)
#
end2 = time.process_time()
end1 = time.perf_counter()


print("CPU time :", end2 - start2, "s")
print("Walltime:", end1 - start1, "s")


CPU time : 1615.125 s
Walltime: 794.9386912999908 s


In [24]:
start1 = time.perf_counter()
start2 = time.process_time()
model2 = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=8, # La taille du "contexte", ici 5 mots avant et après le mot observé (5 mots avant + 5 mots après)
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)
#
end2 = time.process_time()
end1 = time.perf_counter()

print("CPU time :", end2 - start2, "s")
print("Walltime:", end1 - start1, "s")

CPU time : 1692.84375 s
Walltime: 888.8154031000158 s


In [25]:
start1 = time.perf_counter()
start2 = time.process_time()
model3 = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé (5 mots avant + 5 mots après)
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)
#
end2 = time.process_time()
end1 = time.perf_counter()

print("CPU time :", end2 - start2, "s")
print("Walltime:", end1 - start1, "s")

CPU time : 2424.890625 s
Walltime: 1126.4086258999887 s


In [26]:
start1 = time.perf_counter()
start2 = time.process_time()
model4 = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=8, # La taille du "contexte", ici 5 mots avant et après le mot observé (5 mots avant + 5 mots après)
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)
#
end2 = time.process_time()
end1 = time.perf_counter()

print("CPU time :", end2 - start2, "s")
print("Walltime:", end1 - start1, "s")

CPU time : 2574.453125 s
Walltime: 1091.7135911000078 s


In [28]:
models = [model1, model2, model3, model4]

Etape 7: Explorez le modèle retenu à l’aide des fonctions similarity et most_similar (trois exemples pour chaque fonction)

In [48]:
# Explorer le modèle avec similarity
print("Couple de mot fortement lié")
for i in range(len(models)):
    print("modèle " + str(i+1)+ ": " + str(models[i].wv.similarity("roi", "reine")))
    
print("Couple de mot moyennement lié")
for i in range(len(models)):
    print("modèle " + str(i+1) + ": " + str(models[i].wv.similarity("travail", "ouvrier")))

print("Couple de mot négativement lié")
for i in range(len(models)):
    print("modèle " + str(i+1) + ": " + str(models[i].wv.similarity("guerre", "cuisine")))


Couple de mot fortement lié
modèle 1: 0.53389823
modèle 2: 0.51810145
modèle 3: 0.55058235
modèle 4: 0.55070615
Couple de mot moyennement lié
modèle 1: 0.36210838
modèle 2: 0.32590276
modèle 3: 0.37309307
modèle 4: 0.32606763
Couple de mot négativement lié
modèle 1: 0.018428534
modèle 2: -0.08880977
modèle 3: 0.1115908
modèle 4: -0.093228504


In [53]:
# Explorer le modèle avec most_similar
print("Exemple 1(guerre):")
for i in range(len(models)):
    print("modèle " + str(i+1)+ ": ")
    print(models[i].wv.most_similar("guerre", topn=20))
print()
print("Exemple 2(travail):")   
for i in range(len(models)):
    print("modèle " + str(i+1)+ ": ")
    print(models[i].wv.most_similar("travail", topn=10))
print()
print("Exemple 3(mort):")    
for i in range(len(models)):
    print("modèle " + str(i+1)+ ": ")
    print(models[i].wv.most_similar("mort", topn=10))

Exemple 1(guerre):
modèle 1: 
[('liberation', 0.9137529134750366), ('guerro', 0.8887047171592712), ('derniere_guerre', 0.8718257546424866), ('wehrmacht', 0.8489624261856079), ('population', 0.8462672829627991), ('guorre', 0.8428082466125488), ('population_civile', 0.822415292263031), ('guorro', 0.818413496017456), ('guerre_mondiale', 0.8178240060806274), ('separation', 0.8152499794960022), ('colonie', 0.8130805492401123), ('reconnaissance', 0.8119193315505981), ('revolution', 0.8026339411735535), ('communaute', 0.8017407655715942), ('premiere_guerre_mondiale', 0.7986695170402527), ('capitulation', 0.7974178791046143), ('republique_federale', 0.7964221239089966), ('vie_politique', 0.7939084768295288), ('guerre_civile', 0.7934905886650085), ('marine_britannique', 0.7930331230163574)]
modèle 2: 
[('liberation', 0.8935251235961914), ('guerro', 0.8583217263221741), ('derniere_guerre', 0.852436900138855), ('guerre_mondiale', 0.8255516290664673), ('capitulation', 0.81919926404953), ('wehrmach